In [1]:
import pandas as pd
import re
import string
from nltk.corpus import stopwords
import numpy as np
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Flatten
from keras.layers import Embedding
from keras.layers.convolutional import Conv1D
from keras.layers.convolutional import MaxPooling1D
from numpy import asarray
from numpy import zeros
from keras.preprocessing.text import Tokenizer
from keras.utils.np_utils import *
from sklearn.feature_extraction.text import TfidfVectorizer
from gensim.models import Word2Vec
from numpy import array
from pickle import dump
from keras.utils import to_categorical
from keras.layers import Dense
from keras.layers import LSTM
import random
from pickle import load
from keras.models import load_model

Using TensorFlow backend.


# 数据预处理

导入测试和训练组数据

In [2]:
data_train=pd.read_csv('train.csv',header=None)
data_train.columns=['label','article title','article text']
data_test=pd.read_csv('test.csv',header=None)
data_test.columns=['label','article title','article text']

In [4]:
raw_train = []
raw_test = []
for i in range(0,len(data_train['article title'])):
    A = data_train['article text'][i].split()
    raw_train.append(len(A)) 
print('raw train data length:', sum(raw_train))

for i in range(0,len(data_test['article title'])):
    A = data_test['article text'][i].split()
    raw_test.append(len(A)) 
print('raw test data length:', sum(raw_test))

raw train data length: 1243472
raw test data length: 123040


文本清洗

In [5]:
content_train = []
for i in range(0,len(data_train['article title'])):
    A = data_train['article text'][i].split()
    re_punc = re.compile('[%s]' % re.escape(string.punctuation))
    # remove punctuation from each word
    A = [re_punc.sub('', w) for w in A]
    # remove non-printatble characters
    re_print = re.compile('[^%s]'%re.escape(string.printable))
    A = [re_print.sub('', w) for w in A]
    # convert to lower case
    A = [word.lower() for word in A]
    # remove remaining tokens that are not alphabetic
    A = [word for word in A if word.isalpha()]
    # filter out stopwords
    stop_words = set(stopwords.words('english'))
    A = [w for w in A if not w in stop_words]
    # filter out short tokens
    A = [word for word in A if len(word) > 1]
    A = " ".join(A)
    content_train.append(A)

In [6]:
cleaned_train = []
for i in range(0,len(data_train['article title'])):
    A = data_train['article text'][i].split()
    re_punc = re.compile('[%s]' % re.escape(string.punctuation))
    # remove punctuation from each word
    A = [re_punc.sub('', w) for w in A]
    # remove non-printatble characters
    re_print = re.compile('[^%s]'%re.escape(string.printable))
    A = [re_print.sub('', w) for w in A]
    # convert to lower case
    A = [word.lower() for word in A]
    # remove remaining tokens that are not alphabetic
    A = [word for word in A if word.isalpha()]
    # filter out stopwords
    stop_words = set(stopwords.words('english'))
    A = [w for w in A if not w in stop_words]
    # filter out short tokens
    A = [word for word in A if len(word) > 1]
    cleaned_train.append(len(A))
print('cleaned train data length:',sum(cleaned_train))

cleaned train data length: 772325


In [7]:
content_test = []
for i in range(0,len(data_test['article title'])):
    A = data_test['article text'][i].split()
    re_punc = re.compile('[%s]' % re.escape(string.punctuation))
    # remove punctuation from each word
    A = [re_punc.sub('', w) for w in A]
    # remove non-printatble characters
    re_print = re.compile('[^%s]'%re.escape(string.printable))
    A = [re_print.sub('', w) for w in A]
    # convert to lower case
    A = [word.lower() for word in A]
    # remove remaining tokens that are not alphabetic
    A = [word for word in A if word.isalpha()]
    # filter out stopwords
    stop_words = set(stopwords.words('english'))
    A = [w for w in A if not w in stop_words]
    # filter out short tokens
    A = [word for word in A if len(word) > 1]
    A = " ".join(A)
    content_test.append(A)

In [8]:
cleaned_test = []
for i in range(0,len(data_test['article title'])):
    A = data_train['article text'][i].split()
    re_punc = re.compile('[%s]' % re.escape(string.punctuation))
    # remove punctuation from each word
    A = [re_punc.sub('', w) for w in A]
    # remove non-printatble characters
    re_print = re.compile('[^%s]'%re.escape(string.printable))
    A = [re_print.sub('', w) for w in A]
    # convert to lower case
    A = [word.lower() for word in A]
    # remove remaining tokens that are not alphabetic
    A = [word for word in A if word.isalpha()]
    # filter out stopwords
    stop_words = set(stopwords.words('english'))
    A = [w for w in A if not w in stop_words]
    # filter out short tokens
    A = [word for word in A if len(word) > 1]
    cleaned_test.append(len(A)) 
print('cleaned test data length:',sum(cleaned_test))

cleaned test data length: 77721


In [9]:
Class_train = data_train['label']
    
Class_test = data_test['label']

## 根据 TF-IDF 信息训练模型

In [10]:
# create the transform
vectorizer = TfidfVectorizer()
# tokenize and build vocab
vectorizer.fit(content_train)
# summarize
print(vectorizer.vocabulary_)
print(vectorizer.idf_)
# encode document
vector_train = vectorizer.transform(content_train)
vector_test = vectorizer.transform(content_test)
# summarize encoded vector
print(vector_train.shape)
print(vector_train.toarray())

{'ap': 2145, 'washington': 48042, 'wizards': 48937, 'point': 32630, 'guard': 18061, 'steve': 41816, 'blake': 4720, 'miss': 27318, 'first': 15302, 'month': 27642, 'season': 38716, 'injuring': 21256, 'ankle': 1889, 'pickup': 32172, 'game': 16740, 'move': 27902, 'designed': 11053, 'help': 18986, 'better': 4368, 'compete': 8496, 'europe': 13895, 'hewlettpackard': 19120, 'said': 37935, 'acquired': 334, 'synstar': 43015, 'plc': 32498, 'ukbased': 46195, 'technology': 43738, 'services': 39166, 'firm': 15289, 'million': 27104, 'us': 46920, 'airways': 968, 'pilots': 32237, 'voted': 47759, 'approve': 2269, 'new': 28845, 'labor': 23635, 'agreement': 837, 'yesterday': 49502, 'reduce': 36021, 'salaries': 37993, 'percent': 31791, 'save': 38281, 'airline': 944, 'year': 49440, 'uk': 46193, 'panel': 31176, 'examining': 14052, 'radiation': 35307, 'risks': 37229, 'says': 38316, 'official': 29885, 'estimates': 13816, 'dangers': 10210, 'health': 18828, 'may': 26245, 'wide': 48621, 'mark': 25933, 'substantia

(40000, 49798)
[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]


## 随机森林训练

In [1]:
rf = RandomForestClassifier(n_estimators = 500, max_depth = 3, random_state = 123, n_jobs = -1)
rf.fit(vector_train, Class_train)
rfpreds = rf.predict(vector_test)
accuracy_score(Class_test,rfpreds)

NameError: name 'RandomForestClassifier' is not defined

# 根据Word2Vec 训练模型

In [12]:
content_train_wv = []
for i in range(0,len(data_train['article title'])):
    A = data_train['article text'][i].split()
    re_punc = re.compile('[%s]' % re.escape(string.punctuation))
    # remove punctuation from each word
    A = [re_punc.sub('', w) for w in A]
    # remove non-printatble characters
    re_print = re.compile('[^%s]'%re.escape(string.printable))
    A = [re_print.sub('', w) for w in A]
    # convert to lower case
    A = [word.lower() for word in A]
    # remove remaining tokens that are not alphabetic
    A = [word for word in A if word.isalpha()]
    # filter out stopwords
    stop_words = set(stopwords.words('english'))
    A = [w for w in A if not w in stop_words]
    # filter out short tokens
    A = [word for word in A if len(word) > 1]
    content_train_wv.append(A)

In [13]:
content_test_wv = []
for i in range(0,len(data_test['article title'])):
    A = data_test['article text'][i].split()
    re_punc = re.compile('[%s]' % re.escape(string.punctuation))
    # remove punctuation from each word
    A = [re_punc.sub('', w) for w in A]
    # remove non-printatble characters
    re_print = re.compile('[^%s]'%re.escape(string.printable))
    A = [re_print.sub('', w) for w in A]
    # convert to lower case
    A = [word.lower() for word in A]
    # remove remaining tokens that are not alphabetic
    A = [word for word in A if word.isalpha()]
    # filter out stopwords
    stop_words = set(stopwords.words('english'))
    A = [w for w in A if not w in stop_words]
    # filter out short tokens
    A = [word for word in A if len(word) > 1]
    content_test_wv.append(A)

In [15]:
model_train = Word2Vec(content_train_wv, size=100, window=5, workers=4, min_count=1)

In [16]:
# summarize vocabulary size in train model
words = list(model_train.wv.vocab)
print('Train model vocabulary size: %d' % len(words))

Train model vocabulary size: 49798


In [17]:
# save model in ASCII (word2vec) format
filename = 'embedding_word2vec_5.txt'
model_train.wv.save_word2vec_format(filename, binary=False)

In [18]:
# load embedding as a dict
def load_embedding(filename):
# load embedding into memory, skip first line
    file = open(filename,'r')
    lines = file.readlines()[1:]
    file.close()
    # create a map of words to vectors
    embedding = dict()
    for line in lines:
        parts = line.split()
    # key is string word, value is numpy array for vector
        embedding[parts[0]] = asarray(parts[1:], dtype='float32')
    return embedding

In [19]:
# create a weight matrix for the Embedding layer from a loaded embedding
def get_weight_matrix(embedding, vocab):
# total vocabulary size plus 0 for unknown words
    vocab_size = len(vocab) + 1
    # define weight matrix dimensions with all 0
    weight_matrix = zeros((vocab_size, 100))
    # step vocab, store vectors using the Tokenizer's integer mapping
    for word, i in vocab.items():
        weight_matrix[i] = embedding.get(word)
    return weight_matrix

## 随机森林训练

In [20]:
# load embedding from file
raw_embedding = load_embedding('embedding_word2vec_5.txt')
content_train_rf = content_train_wv

# replace each word to their corresponding word2vec embedding.
for i in range(0,len(content_train_rf)):
    for x in range(0,len(content_train_rf[i])):
        content_train_rf[i][x] = raw_embedding[content_train_rf[i][x]]

In [21]:
# sum up and creat an overall vector for each news
B = np.zeros(100)
for i in range(0,len(content_train_rf)):
    for x in range(0,len(content_train_rf[i])):
        B = B + np.array(content_train_rf[i][x])
    content_train_rf[i] = B
    B = np.zeros(100)

In [22]:
# check whether this word that caused bug exists in the testing data.
B = [i for j in content_test_wv for i in j] ;
if 'munoz' in B:
    print('1')
else:
    print('not exist')

1


In [23]:
# load embedding from file, for words that exists in testing data but not testing data, assign a zero vector to them.
content_test_rf = content_test_wv
for i in range(0,len(content_test_rf)):
    for x in range(0,len(content_test_rf[i])):
        content_test_rf[i][x] = raw_embedding.get(content_test_rf[i][x],zeros(100))

In [24]:
C = np.zeros(100)
for i in range(0,len(content_test_rf)):
    for x in range(0,len(content_test_rf[i])):
        C = C + np.array(content_test_rf[i][x])
    content_test_rf[i] = C
    C = np.zeros(100)

In [25]:
rf1 = RandomForestClassifier(n_estimators = 500, max_depth = 3, random_state = 123, n_jobs = -1)
rf1.fit(content_train_rf, Class_train)
rfpreds_wv = rf1.predict(content_test_rf)
accuracy_score(Class_test,rfpreds_wv)

0.801

## CNN训练

In [57]:
# create the tokenizer
tokenizer = Tokenizer()
# fit the tokenizer on the documents
tokenizer.fit_on_texts(content_train) 
# sequence encode
encoded_train = tokenizer.texts_to_sequences(content_train)
# pad sequences
max_length= max([len(s.split()) for s in content_train])
Xtrain = pad_sequences(encoded_train, maxlen=max_length, padding='post')
# define training labels
ytrain = to_categorical(Class_train)
# do the same for testing data
encoded_test = tokenizer.texts_to_sequences(content_test)
Xtest = pad_sequences(encoded_test, maxlen=max_length, padding='post')
ytest = to_categorical(Class_test,num_classes=5)

In [28]:
# define vocabulary size (largest integer value)
vocab_size = len(tokenizer.word_index) + 1
# get vectors in the right order
embedding_vectors = get_weight_matrix(raw_embedding, tokenizer.word_index)
# create the embedding layer
embedding_layer = Embedding(vocab_size, 100, weights=[embedding_vectors], 
                            input_length=max_length, trainable=False)
# define model
model_dl = Sequential()
model_dl.add(embedding_layer)
model_dl.add(Conv1D(filters=32, kernel_size=5, activation='relu'))
model_dl.add(MaxPooling1D(pool_size=2))
model_dl.add(Flatten())
model_dl.add(Dense(5, activation='softmax'))
print(model_dl.summary())
# compile network
model_dl.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
# fit network
model_dl.fit(Xtrain, ytrain, epochs=10, verbose=2)
# evaluate
loss, acc = model_dl.evaluate(Xtest, ytest, verbose=0)
print('Test Accuracy: %f' % (acc*100))

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 85, 100)           4979900   
_________________________________________________________________
conv1d_1 (Conv1D)            (None, 81, 32)            16032     
_________________________________________________________________
max_pooling1d_1 (MaxPooling1 (None, 40, 32)            0         
_________________________________________________________________
flatten_1 (Flatten)          (None, 1280)              0         
_________________________________________________________________
dense_1 (Dense)              (None, 5)                 6405      
Total params: 5,002,337
Trainable params: 22,437
Non-trainable params: 4,979,900
_________________________________________________________________
None
Epoch 1/10
 - 7s - loss: 0.5214 - accuracy: 0.8144
Epoch 2/10
 - 7s - loss: 0.4567 - accuracy: 0.834

In [29]:
%%time
rf.fit(vector_train, Class_train)

CPU times: user 5.38 s, sys: 491 ms, total: 5.87 s
Wall time: 2.27 s


RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
                       max_depth=3, max_features='auto', max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=500,
                       n_jobs=-1, oob_score=False, random_state=123, verbose=0,
                       warm_start=False)

In [30]:
%%time
rf1.fit(content_train_rf, Class_train)

CPU times: user 1min 23s, sys: 232 ms, total: 1min 23s
Wall time: 21.9 s


RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
                       max_depth=3, max_features='auto', max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=500,
                       n_jobs=-1, oob_score=False, random_state=123, verbose=0,
                       warm_start=False)

In [31]:
%%time
model_dl.fit(Xtrain, ytrain, epochs=10, verbose=2)

Epoch 1/10
 - 7s - loss: 0.3679 - accuracy: 0.8664
Epoch 2/10
 - 7s - loss: 0.3637 - accuracy: 0.8689
Epoch 3/10
 - 7s - loss: 0.3578 - accuracy: 0.8690
Epoch 4/10
 - 7s - loss: 0.3558 - accuracy: 0.8705
Epoch 5/10
 - 7s - loss: 0.3525 - accuracy: 0.8709
Epoch 6/10
 - 7s - loss: 0.3483 - accuracy: 0.8735
Epoch 7/10
 - 7s - loss: 0.3456 - accuracy: 0.8736
Epoch 8/10
 - 7s - loss: 0.3443 - accuracy: 0.8742
Epoch 9/10
 - 7s - loss: 0.3396 - accuracy: 0.8756
Epoch 10/10
 - 7s - loss: 0.3390 - accuracy: 0.8742
CPU times: user 2min 40s, sys: 15.1 s, total: 2min 55s
Wall time: 1min 6s


# 文本生成

In [32]:
#load 'sport' training data
data_sport = list(data_train.loc[data_train['label']==2]['article text'])

In [33]:
sport_generation = []
for i in range(0,len(data_sport)):
    A = data_sport[i].split()
    re_punc = re.compile('[%s]' % re.escape(string.punctuation))
    # remove punctuation from each word
    A = [re_punc.sub('', w) for w in A]
    # remove non-printatble characters
    re_print = re.compile('[^%s]'%re.escape(string.printable))
    A = [re_print.sub('', w) for w in A]
    # convert to lower case
    A = [word.lower() for word in A]
    # remove remaining tokens that are not alphabetic
    A = [word for word in A if word.isalpha()]
    # filter out stopwords
    stop_words = set(stopwords.words('english'))
    A = [w for w in A if not w in stop_words]
    # filter out short tokens
    A = [word for word in A if len(word) > 1]
    A = " ".join(A)
    sport_generation.append(A)

In [34]:
# integer encode sequences of words
tokenizer_text = Tokenizer()
tokenizer_text.fit_on_texts(sport_generation)
sequences = tokenizer_text.texts_to_sequences(sport_generation)
# vocabulary size
vocab_size_text = len(tokenizer_text.word_index) + 1

In [35]:
vocab_size_text

19879

In [36]:
# separate text into input and output
X=[]
y=[]
for i in range(0,len(sequences)):
    X.append(sequences[i][:-1])
    y.append(sequences[i][-1])

In [37]:
# padding each input to the longest input length
max_input= max([len(s) for s in X])
X = pad_sequences(X, maxlen=max_input, padding='pre')
y = to_categorical(y, num_classes=vocab_size_text)

In [38]:
# get vectors in the right order
embedding_vectors_text = get_weight_matrix(raw_embedding, tokenizer_text.word_index)
# create the embedding layer
embedding_layer_text = Embedding(vocab_size_text, 100, weights=[embedding_vectors_text], 
                                 input_length=max_input, trainable=True)
# define model
model_text = Sequential()
model_text.add(embedding_layer_text)
model_text.add(LSTM(100))
model_text.add(Dense(vocab_size_text, activation='softmax'))
print(model_text.summary())
# compile network
model_text.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
# fit network
model_text.fit(X, y, epochs=30, verbose=1)
# save the model to file
model_text.save('model_text1.h5')
# save the tokenizer
dump(tokenizer_text, open('tokenizer_text1.pkl', 'wb'))


Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_2 (Embedding)      (None, 79, 100)           1987900   
_________________________________________________________________
lstm_1 (LSTM)                (None, 100)               80400     
_________________________________________________________________
dense_2 (Dense)              (None, 19879)             2007779   
Total params: 4,076,079
Trainable params: 4,076,079
Non-trainable params: 0
_________________________________________________________________
None


/opt/anaconda3/lib/python3.7/site-packages/tensorflow_core/python/framework/indexed_slices.py:433: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


Epoch 1/30
10000/10000 [==============================] - 43s 4ms/step - loss: 8.1987 - accuracy: 0.0277
Epoch 2/30
10000/10000 [==============================] - 43s 4ms/step - loss: 7.0975 - accuracy: 0.0467
Epoch 3/30
10000/10000 [==============================] - 42s 4ms/step - loss: 6.7214 - accuracy: 0.0599
Epoch 4/30
10000/10000 [==============================] - 42s 4ms/step - loss: 6.2811 - accuracy: 0.0767
Epoch 5/30
10000/10000 [==============================] - 42s 4ms/step - loss: 5.8346 - accuracy: 0.0915
Epoch 6/30
10000/10000 [==============================] - 42s 4ms/step - loss: 5.4045 - accuracy: 0.1183
Epoch 7/30
10000/10000 [==============================] - 42s 4ms/step - loss: 4.9678 - accuracy: 0.1513
Epoch 8/30
10000/10000 [==============================] - 42s 4ms/step - loss: 4.5369 - accuracy: 0.1897
Epoch 9/30
10000/10000 [==============================] - 42s 4ms/step - loss: 4.1120 - accuracy: 0.2376
Epoch 10/30
10000/10000 [==============================

In [39]:
# load model
model_text = load_model('model_text1.h5')
# load the tokenizer
tokenizer_text = load(open('tokenizer_text1.pkl', 'rb'))

/opt/anaconda3/lib/python3.7/site-packages/tensorflow_core/python/framework/indexed_slices.py:433: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


In [40]:
# generate a sequence from a language model
def generate_seq(model, tokenizer, seq_length, seed_text, n_words):
    result = list()
    in_text = seed_text
    # generate a fixed number of words
    for _ in range(n_words):
        # encode the text as integer
        encoded = tokenizer.texts_to_sequences([in_text])[0]
        # truncate sequences to a fixed length
        encoded = pad_sequences([encoded], maxlen=seq_length, truncating='pre')
        # predict probabilities for each word
        yhat = model.predict_classes(encoded, verbose=0)
        # map predicted word index to word
        out_word = ''
        for word, index in tokenizer.word_index.items():
            if index == yhat:
                out_word = word
                break
        # append to input
        in_text += ' ' + out_word
        result.append(out_word)
    return ' '.join(result)

In [41]:
# generate 100 samples
sample_texts = []
for i in range(0,100):
    seed_text = sport_generation[random.randint(0,len(sport_generation))]
    generated = generate_seq(model_text, tokenizer_text, max_input, seed_text, 85)
    sample_texts.append(generated)
print(sample_texts)

['touched watching victory lap victory twins indians indians st supersonics st supersonics pm supersonics pm supersonics pm supersonics pm supersonics pm supersonics pm supersonics pm supersonics pm supersonics pm supersonics pm supersonics pm supersonics pm supersonics spurs pm braves classic st supersonics classic st spurs park st supersonics pm supersonics pm supersonics pm supersonics pm supersonics spurs pm braves classic st supersonics classic st spurs park st st supersonics pm supersonics pm supersonics pm supersonics pm supersonics pm supersonics pm supersonics pm supersonics pm supersonics', 'eyes easy enough enough enough enough bowl victory detroit detroit detroit detroit sox sox sox sox sox sox sox sox sox sox sox sox sox sox sox sox sox sox sox sox sox sox sox sox sox sox sox sox sox sox sox sox sox sox sox sox sox sox sox sox sox sox sox sox sox sox sox sox sox sox sox sox sox sox sox sox sox sox sox sox sox sox sox sox sox sox sox sox sox sox sox monday night', 'field fi

## 测试 基于TF-IDF训练的随机森林模型

In [42]:
text_test = []
for i in range(0,100):
    text_test.append(2)
text_test = np.array(text_test)

In [43]:
generated_tfidf = sample_texts
# encode document
vector_test_text = vectorizer.transform(generated_tfidf)
rfpreds_text = rf.predict(vector_test_text)
accuracy_score(text_test,rfpreds_text)

0.83

## 测试 基于Word2vec训练的随机森林模型

In [44]:
generated_rf = sample_texts
generated_rf_wv = []

for i in range(0,len(generated_rf)):
    A = generated_rf[i].split()
    generated_rf_wv.append(A)

In [45]:
for i in range(0,len(generated_rf_wv)):
    for x in range(0,len(generated_rf_wv[i])):
        generated_rf_wv[i][x] = raw_embedding.get(generated_rf_wv[i][x],zeros(100))

In [46]:
D = np.zeros(100)
for i in range(0,len(generated_rf_wv)):
    for x in range(0,len(generated_rf_wv[i])):
        D = D + np.array(generated_rf_wv[i][x])
    generated_rf_wv[i] = D
    D = np.zeros(100)

In [47]:
rfpreds_rf = rf1.predict(generated_rf_wv)
accuracy_score(text_test,rfpreds_rf)

0.81

## 测试 基于Word2vec训练的CNN模型

In [48]:
generated_dl = sample_texts

In [49]:
dl_test = tokenizer.texts_to_sequences(generated_dl)
dl_test = pad_sequences(dl_test, maxlen=85, padding='post')

In [50]:
dl_class = []
for i in range(0,100):
    dl_class.append([0,0,1,0,0])
dl_class = np.array(dl_class)

In [51]:
# evaluate
loss, acc = model_dl.evaluate(dl_test, dl_class, verbose=0)
print('Test Accuracy: %f' % (acc*100))

Test Accuracy: 20.000000
